# Synthetic Radio Host Generator

# Step 1: Repository Setup


In [ ]:
import os
import sys
import subprocess
import logging

REPO_URL = "https://github.com/BoteAkshay/synthetic_radio_host.git"
REPO_DIR = "/content/synthetic_radio_host"
BRANCH_NAME = "main"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

if not os.path.exists(REPO_DIR):
    logger.info(f"Cloning repository from {REPO_URL}")
    subprocess.run(["git", "clone", "--branch", BRANCH_NAME, "--single-branch", REPO_URL, REPO_DIR], check=True)
else:
    logger.info("Updating repository")
    subprocess.run(["git", "fetch", "origin"], cwd=REPO_DIR, check=True)
    subprocess.run(["git", "checkout", BRANCH_NAME], cwd=REPO_DIR, check=True)
    subprocess.run(["git", "pull", "origin", BRANCH_NAME], cwd=REPO_DIR, check=True)

if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)


# Step 2: Dependency Installation


In [ ]:
REPO_SRC_DIR = "/content/synthetic_radio_host/src"
REQUIREMENTS_FILE = "/content/synthetic_radio_host/requirements.txt"
DEPS_MARKER = "/content/.deps_installed"

if not os.path.exists(DEPS_MARKER):
    logger.info("Installing dependencies")
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", REQUIREMENTS_FILE], check=True)
    open(DEPS_MARKER, "w").close()
    logger.info("Dependencies installed")
else:
    logger.info("Dependencies already installed")

if REPO_SRC_DIR not in sys.path:
    sys.path.insert(0, REPO_SRC_DIR)

2026-01-02 17:13:39 - INFO - Dependencies already installed — skipping
2026-01-02 17:13:39 - INFO - Added /content/synthetic_radio_host/src to Python path


# Step 3: Import Libraries and Initialize API Clients

In [ ]:
from radio_host_functions import (
    CONFIG,
    fetch_wikipedia_article,
    generate_script_prompt,
    generate_script,
    generate_audio_segments,
    combine_and_export_audio
)
from openai import OpenAI
from google.colab import userdata, files

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
ELEVENLABS_API_KEY = userdata.get("ELEVENLABS_API_KEY")

if not OPENAI_API_KEY or not ELEVENLABS_API_KEY:
    raise ValueError("API keys not found in Colab secrets")

openai_client = OpenAI(api_key=OPENAI_API_KEY)
os.environ["ELEVENLABS_API_KEY"] = ELEVENLABS_API_KEY

try:
    from elevenlabs.client import ElevenLabs
    eleven_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
except ModuleNotFoundError:
    import elevenlabs
    elevenlabs.set_api_key(ELEVENLABS_API_KEY)
    eleven_client = elevenlabs

logger.info("API clients initialized")



# Step 4: Fetch Wikipedia Article


In [ ]:
#Below topic can be changed
WIKIPEDIA_TOPIC = "MS Dhoni"

logger.info(f"Fetching Wikipedia article: {WIKIPEDIA_TOPIC}")
wiki_text = fetch_wikipedia_article(WIKIPEDIA_TOPIC)
logger.info(f"Fetched article | Characters: {len(wiki_text)}")



# Step 5: Generate Script


In [ ]:
MIN_SCRIPT_CHARS = 600
MAX_RETRIES = 2

logger.info("Generating script prompt")
hinglish_prompt = generate_script_prompt(wiki_text)

script = None
for attempt in range(1, MAX_RETRIES + 1):
    try:
        logger.info(f"Generating script (attempt {attempt}/{MAX_RETRIES})")
        script = generate_script(hinglish_prompt, openai_client)
        
        if script and len(script) >= MIN_SCRIPT_CHARS:
            logger.info(f"Script generated | Characters: {len(script)}")
            break
    except Exception as e:
        if attempt == MAX_RETRIES:
            raise
        logger.warning(f"Attempt {attempt} failed: {e}")

if not script or len(script) < MIN_SCRIPT_CHARS:
    raise RuntimeError(f"Script generation failed or too short ({len(script) if script else 0} chars)")


# Step 6: Generate Audio Segments


In [ ]:
logger.info("Generating audio segments")
audio_segments = generate_audio_segments(script)
logger.info(f"Audio segments generated | Count: {len(audio_segments)}")


# Step 7: Combine and Export Audio


In [ ]:
from google.colab import files

OUTPUT_FILE = CONFIG.get("OUTPUT_FILENAME")

logger.info("Combining and exporting audio")
combine_and_export_audio(audio_segments, OUTPUT_FILE)

file_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
logger.info(f"Audio file created | File: {OUTPUT_FILE} | Size: {file_size_mb:.2f} MB")

files.download(OUTPUT_FILE)